# BBVDATA Preprocessing

### Импортирование инструментов и инициализация предобработки

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("telemetrika_db.csv", header=0)

FileNotFoundError: File b'telemetrika_db.csv' does not exist

In [ ]:
df.head()

In [ ]:
df.info()

#### Исходные данные:
- uid – [тип данных: uuid, представляет собой 16-байтный (128-битный) номер В шестнадцатеричной системе счисления: d4372bfe-1738-440e-9121-6f96302a98c4
- bv_number – тип данных: smallint, значение- номер камеры, пример: 10.
- speed - тип данных: smallint, значение – средняя скорость потока, пример: 60.
- load – тип данных: smallint, значение – загруженность дороги в процентах от 0 до 100
- amount – тип данных: smallint, значение – количество автомашин за период времени, пример: 143
- image_name – тип данных: character varying(40), имя файла в папке расположения.
- date – тип данных: timestamp(6) with time zone, дата и время. Пример: "2018-05-24 18:26:10+06"
- point – тип данных: smallint, бальность потока. от 1 до 10 (зависит от скорости потока и загруженности дороги)
- time – тип данных: bigint, unix timestamp, пример: 1527164746000
- id – bigint, serial. Внутренняя идентификатор записей в таблице

#### Искомые данные на вывод:
- Прогноз количества автотранспорта в определенных точках и загруженности дорог

#### Факторы влияющие на прогноз:
- Погода
- Праздники

### Удаление столбцов, которые не влияют на обучение или дублируются

In [ ]:
bbvdata = df.drop([
                    "uid", 
                    "image_name",  
                    "time", 
                    "id"
                  ], axis=1)\
                  [[
                    "amount", 
                    "load", 
                    "point",
                    "speed",                                           
                    "bv_number", 
                    "date"
                  ]]
bbvdata.head()

Так как значение amount (количество автомашин) за определенный период времени не может быть отрицательным, не рассматриваем **1728** строк данных 

In [ ]:
print(sum(bbvdata["amount"] < 0))
bbvdata = bbvdata[bbvdata["amount"] > 0]

In [ ]:
bbvdata.isnull().sum()

In [ ]:
sns.heatmap(bbvdata.corr(), cmap="coolwarm")

По тепловой карте корреляции видно, что поля **load** и **point** являются равнозначными и имеют одинаковое поведение по отношению к выходным данным.

Видно, что корреляция поля **speed** по отношению к другим полям имеется(*amount*), но процент влияния меньше 40% и имеет смежную негативную корреляцию с **load** и **point**, так что имеет смысл не рассматривать это поле.

In [ ]:
bbvdata = bbvdata.drop([
                    "point",
                    "speed"
                  ], axis=1)\
                  [[
                    "amount", 
                    "load",                                          
                    "bv_number", 
                    "date"
                  ]]
bbvdata.head()

### Дополнительные поля 

Так как загруженность на дорогах тесно связано с определенными периодами времени, имеет смысл разбить данные по месяцам, дням недели и по временным отрезкам. 

Мы можем агрегировать данные по временным отрезкам и поделить 24 часа на 144 сектора, где периодичностью изменений является 10 минут.

In [ ]:
bbvdata["date"] = pd.to_datetime(bbvdata["date"])

In [ ]:
bbvdata["Year"] = bbvdata["date"].apply(lambda d: d.year)
bbvdata["Hour"] = bbvdata["date"].apply(lambda d: d.hour)
bbvdata["Month"] = bbvdata["date"].apply(lambda d: d.month)
bbvdata["Chunk"] = bbvdata["date"].apply(lambda d: int((d.hour*60+d.minute)/10))
bbvdata["Day of the Week"] = bbvdata["date"].apply(lambda d: d.weekday()).map({
    0: 'Mon',
    1: 'Tue',
    2: 'Wed',
    3: 'Thu',
    4: 'Fri',
    5: 'Sat',
    6: 'Sun'
})

In [ ]:
sns.countplot(x="Year", data=bbvdata, hue="bv_number")
plt.legend(bbox_to_anchor=(1.05, 1))

In [ ]:
bbvdata["Year"].value_counts()

На графике хорошо прослеживается контраст. Количество данных за 2016 год небольшой, и имеет смысл не рассматривать эти данные, так как не так сильно влияют на обучение

In [ ]:
bbvdata = bbvdata[bbvdata["Year"] > 2016]

### Слияние с погодой 

Так как погодные явления сильно влияют на движение на дорогах, используем погоду как параметры, где 
- **T** - температура
- **W1** - состояние погоды
- **RRR** - дождь
- **sss** - снег

In [ ]:
weather = pd.read_excel("weather.xlsx")

In [ ]:
weather = weather[['Местное время в Алматы', 'T', 'W1', 'RRR', 'sss']]

In [ ]:
weather.head()

In [ ]:
weather.info()

In [ ]:
weather.isnull().sum()

In [ ]:
weather = weather[~weather.isnull()["T"]]

In [ ]:
weather_states = [
    'Туман или ледяной туман или сильная мгла.',
    'Снег или дождь со снегом.',
    'Ливень (ливни).',
    'Ливни или перемещающиеся осадки.',
    'Дождь.',
    'Осадки',
    'Явление, связанное с переносом ветром твердых частиц, видимость пониженная.',
    'Гроза (грозы) с осадками или без них.',
    'Морось.',
    'Облака покрывали более половины неба в течение всего соответствующего периода.',
    'Облака покрывали более половины неба в течение одной части соответствующего периода и половину или менее в течение другой части периода.',
    'Облака покрывали половину неба или менее в течение всего соответствующего периода.'
]

In [ ]:
weather = weather.replace(np.nan, 0)

In [ ]:
weather['W1'] = weather['W1'].apply(lambda w: 100-((weather_states.index(w) if type(w) != int else w) * 8))

In [ ]:
weather['RRR'] = weather['RRR'].replace("Следы осадков", 1.0)
weather['sss'] = weather['sss'].replace("Менее 0.5", 0.5).replace("Снежный покров не постоянный.", 0.3)

In [ ]:
weather = weather[pd.to_datetime(weather['Местное время в Алматы']).dt.year > 2016]
weather['date'] = pd.to_datetime(weather['Местное время в Алматы']).apply(lambda d: datetime(d.year, d.month, d.day, d.hour))

In [ ]:
from datetime import datetime, timedelta
import time

In [ ]:
quarters = np.repeat(np.arange(0, 22, 3), 3)

In [ ]:
bbvdata["date"] = bbvdata["date"].apply(lambda d: datetime(d.year, d.month, d.day, quarters[d.hour]))

In [ ]:
bbvdata = bbvdata.merge(weather.drop("Местное время в Алматы",axis=1), on='date', how='inner')

In [ ]:
bbvdata = bbvdata.drop(["date", "Hour", "Year"], axis=1)

In [ ]:
bbvdata.head()

In [ ]:
sns.heatmap(bbvdata.groupby(["Day of the Week","Chunk"]).max()["load"].unstack(),cmap="coolwarm")

In [ ]:
sns.heatmap(bbvdata.groupby(["Day of the Week","Chunk"]).mean()["load"].unstack(),cmap="coolwarm")

In [ ]:
X = pd.get_dummies(bbvdata, columns=["bv_number", "Day of the Week"])

In [ ]:
X.shape

In [ ]:
X.to_csv("preprocessed.csv")